In [387]:
import numpy as np
import sqlite3
from scipy.spatial import distance #https://docs.scipy.org/doc/scipy/reference/generated/scipy.spatial.distance.cdist.html


In [388]:
conn = sqlite3.connect('../livs.db')  # Create db and establish connection
conn.row_factory = sqlite3.Row
curs = conn.cursor()
result = []
rows = curs.execute('select * from livs where "Huvudgrupp" = "Grönsaker"')
for row in rows:
        result.append(row)

db_contents = np.array(result)

conn.close()

## Prototypiska i sin kategori
Syftet är att ta en huvudgrupp och se vad mittpunkten i den kategorin är och sen räkna ut vilka livsmedel som ligger närmast mittpunkten.

Varje näringsvärde blir en dimension, och varje livsmedel en punkt som har ett värde i alla de dimensionerna.

Medelvärdet av alla livsmedel blir en punkt som man kan utgå från när man räknar avstånd till alla livsmedlen, och när man har tillgång till avstånden så kan man ordna alla livsmedlen efter avståndet till mittpunkten.

Det är första steget. 

Andra steget är att se om man verkligen behöver alla näringsvärden för att ändå få samma ordning mellan livsmedlen. Vilka kan man ta bort?

### Utgångspunkter

Från Livsmedelsverkets data har vi valt en huvudgrupp, som "Grönsaker". Alla dess data ligger i db_contents, och den variabeln har en shape som säger hur många rader (livsmedel) och kolumner (namn, nummer, näringsvärden, klassificering) den innehåller.

In [389]:
db_contents.shape

(94, 63)

Kolumn 0 är namnet, 1 är numret och sen börjar näringsvärdena. 

columns är variabeln som innehåller de värden som används i beräkningarna. Under står de kolumner som är möjliga att använda. 

Vill man ha fram betydelsena av kolumnerna kan man göra något i stil med

```
for row in conn.execute('PRAGMA table_info (livs);'):
    columnObject.append(row)
columns = np.array(columnObject)
``` 

In [390]:
columns = np.array([ 2,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 35, 37, 38, 40, 41, 42, 43, 44, 45, 46, 47, 48, 50, 51, 52, 53, 54, 55, 56, 57])
# Possible columns: [ 2,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 35, 37, 38, 40, 41, 42, 43, 44, 45, 46, 47, 48, 50, 51, 52, 53, 54, 55, 56, 57]


dataset blir de kolumner som används. Från databasen brukar de komma som unicode, så det är lämpligt att konvertera till float.

In [391]:
dataset = db_contents[:,columns].astype(float)

Det är fantastiskt enkelt att räkna medelvärdet av varje kolumn med funktionen numpy.mean, som returnerar en array med lika många element som antal kolumner

Obs: axis behövs, för annars får man fel medelvärde...

In [392]:
means = np.mean(dataset, axis=0)

In [393]:
print(means, '\n', means.shape)

[  4.23425532e+01   5.17563830e+00   1.07510638e+00   1.88117021e+00
   2.17787234e+00   8.85922340e+01   0.00000000e+00   1.10702128e+00
   1.91085106e+00   9.59574468e-01   9.27659574e-01   5.31914894e+00
   1.81063830e-01   5.85106383e-03   8.51063830e-04   3.08510638e-03
   1.44361702e-01   1.73404255e-02   2.34042553e-03   5.06808511e-01
   3.12765957e-02   4.68297872e-01   2.31595745e-01   1.53085106e-01
   7.08510638e-02   1.06382979e-04   1.06382979e-04   0.00000000e+00
   0.00000000e+00   7.65957447e-02   6.98819149e+01   4.14893617e-03
   9.41489362e-01   5.75531915e-02   6.88297872e-02   3.33329787e+01
   6.17127660e-01   9.29148936e-01   1.35106383e-01   3.72340426e-03
   5.61191489e+01   4.06382979e+01   7.74893617e-01   5.13489362e+01
   2.59646809e+02   1.94946809e+01   1.35105319e+02   3.39255319e-01
   8.05957447e-01   3.57021277e-01] 
 (50,)


Det är läskigt lätt även att räkna avståndet mellan alla punkterna i dataset och medelvärdena:

(Det är något med medelvärdena som gör att man måste sätta en extra klammer runt, för att dimensionaliteten ska stämma...)

In [394]:
dist = distance.cdist(dataset, [means], 'euclidean') #https://docs.scipy.org/doc/scipy/reference/generated/scipy.spatial.distance.cdist.html

*dist* innehåller sen varje rads avstånd från medelvärdet i alla kolumner.

In [395]:
dist

array([[  410.59087183],
       [  131.18395531],
       [  179.80564576],
       [  154.45897082],
       [  204.85406506],
       [  152.11235007],
       [  220.83451466],
       [  198.73915097],
       [  161.25814709],
       [  174.74065684],
       [  222.23458885],
       [  274.91215065],
       [  198.73997875],
       [  189.94233315],
       [  173.89129257],
       [  130.06019898],
       [  170.69576887],
       [  172.28324477],
       [  218.91458608],
       [  202.90275213],
       [  226.01199529],
       [  186.89947764],
       [  332.10591064],
       [  232.4206282 ],
       [  234.97257235],
       [  702.19745614],
       [  166.12027779],
       [  140.4534426 ],
       [  150.28240145],
       [  174.00902005],
       [  117.7339104 ],
       [  183.82231746],
       [  417.39795507],
       [  228.06150751],
       [  191.02168039],
       [  152.8582544 ],
       [  393.66218809],
       [  216.11685089],
       [  154.330754  ],
       [  294.22196039],


Jag har svårt att vänja mig vid det här med implicit sortering.

Först sorterar vi alltså värdena och får en "ordning", som vi sen använder som index för den array vi egentligen vill sortera...

In [396]:
order = np.argsort(dist.T[0])

In [397]:
order

array([62, 57, 68, 30, 81, 15, 89,  1, 70, 77, 71, 27, 93, 60, 82, 64, 28,
        5, 35, 88, 38,  3, 85, 66, 49,  8, 41, 59, 26, 63, 90, 61, 16, 40,
       17, 86, 14, 29,  9, 52,  2, 58, 31, 21, 13, 34, 48, 67, 78,  7, 12,
       92, 56, 83, 19,  4, 47, 37, 18,  6, 10, 79, 20, 33, 23, 55, 24, 50,
       51, 84, 46, 43, 45, 91, 11, 39, 80, 22, 44, 65, 36,  0, 32, 72, 87,
       69, 42, 25, 73, 74, 54, 76, 53, 75])

Eftersom vi har vår ursprungliga lista med oss i *db_contents* så kan vi skriva ut vilka kolumner som helst, och nu med raderna i rätt ordning utifrån medelvärdet i huvudgruppen.

Här tog det ett tag för mig att förstå hur jag skulle se på resultatet. Att se de första som näringsmässigt "lika" var inte så svårt, men hur är det med de livsmedel som hamnar sist i listan?

Jag tror att man ska se det som en (mångdimensionell) "periferi" där de sista i listan förstås är långt från centrum, men inte alls inbördes "lika". De kan ha stort avstånd till mitten i väldigt skilda dimensioner. Det enda som förenar dem är att de är udda... :-)

In [398]:
db_contents[order,0]

array(['Blomkål kokt', 'Tomater hela konserv m lag', 'Purjolök kokt',
       'Savojkål', 'Fänkål kokt', 'Kronärtskocka', 'Kruksallat',
       'Stjälkselleri', 'Vitkål kokt', 'Kronärtskocka kokt',
       'Aubergine kokt', 'Purjolök', 'Brysselkål kokt',
       'Tomater krossade konserv m lag', 'Romansallat',
       'Gurka färskinlagd', 'Rödkål', 'Broccoli fryst', 'Tomat',
       'Röd mangold småbladig färsk', 'Vitkål', 'Blomkål fryst',
       'Småtomater röda typ körsbärstomat', 'Lök kokt',
       'Grönsallat gurka tomat u dressing', 'Friséesallat',
       'Sparris grön el vit', 'Tomater gröna syltade', 'Pumpa',
       'Grönsallat gurka tomat paprika u dressing', 'Lök röd',
       'Pumpasallad m paprika inlagd sötad u lag', 'Kålrabbi', 'Aubergine',
       'Lök gul', 'Broccoli kokt', 'Isbergssallat', 'Huvudsallat',
       'Endivesallat', 'Majskorn konserv u lag', 'Blomkål',
       'Syltlök inlagd', 'Salladskål', 'Mangold', 'Gurka', 'Squash',
       'Lök gul fryst', 'Paprika förvälld', 'Ma

## Del två. Att jämföra ordningar...

Den här ordningen mellan livsmedel bygger på att man tar ett urval kolumner. Typiskt tar man så mycket information man har (= alla kolumner) för att få en mer relevant ordning.

Men kanske är det så att inte alla kolumner behövs för att ge den här ordningen? Vilka kolumner skulle man kunna ta bort?

Först lite repetition av praktiska funktioner i numpy. 

Om man har två arrayer med samma tal fast i olika ordning så kan man jämföra dem genom att ta den ena minus den andra och kolla om det finns några värden som inte är noll. Antalet värden som är skilda från noll blir ett mått på hur olika ordningarna är.

In [400]:
a = np.array([1,2,3,4,5])
b = np.array([1,3,2,4,5])

In [405]:
a-b # Är de lika får man en array med bara nollor

array([ 0, -1,  1,  0,  0])

In [406]:
np.count_nonzero(a-b) 

2

Dags att summera ihop mycket av del 1 i en funktion. Utgå från db_contents som typ en global variabel. Skicka in kolumner till funktionen, så returneras en ordning mellan livsmedlen i db_contents/dataset

In [407]:
def get_order(columns):
    dataset = db_contents[:,columns].astype(float)
    means = np.mean(dataset, axis=0)
    dist = distance.cdist(dataset, [means], 'euclidean')
    order = np.argsort(dist.T[0])
    return order

In [408]:
order_0 = get_order(columns)

In [409]:
order_0

array([62, 57, 68, 30, 81, 15, 89,  1, 70, 77, 71, 27, 93, 60, 82, 64, 28,
        5, 35, 88, 38,  3, 85, 66, 49,  8, 41, 59, 26, 63, 90, 61, 16, 40,
       17, 86, 14, 29,  9, 52,  2, 58, 31, 21, 13, 34, 48, 67, 78,  7, 12,
       92, 56, 83, 19,  4, 47, 37, 18,  6, 10, 79, 20, 33, 23, 55, 24, 50,
       51, 84, 46, 43, 45, 91, 11, 39, 80, 22, 44, 65, 36,  0, 32, 72, 87,
       69, 42, 25, 73, 74, 54, 76, 53, 75])

In [334]:
columns_1 = np.delete(columns,3)

In [335]:
columns_1

array([ 2,  4,  5,  7,  8,  9, 10, 11, 12, 13, 14, 16, 17, 18, 19, 20, 21,
       22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 35, 37, 38, 40, 41,
       42, 43, 44, 45, 46, 47, 48, 50, 51, 52, 53, 54, 55, 56, 57])

In [336]:
columns

array([ 2,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 16, 17, 18, 19, 20,
       21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 35, 37, 38, 40,
       41, 42, 43, 44, 45, 46, 47, 48, 50, 51, 52, 53, 54, 55, 56, 57])

In [337]:
order_1 = get_order(columns_1)

In [338]:
x = np.count_nonzero(order_0-order_1)

In [339]:
x

0

In [375]:
columns

array([ 2,  4,  8, 11, 14, 35, 38, 42, 47, 48, 51, 52])

In [376]:
size = columns.size
print(size)
a = np.empty([size,size-1],dtype=int)
print(a)
for index, item in enumerate(columns):
    a[index]=np.delete(columns,index)
    print(a)

12
[[8070450532247928832 8070450532247928832          4458718616
           4458443384          4462987048          4458443864
           4482558776          4482558456          4480528856
           4480531576          4480476568]
 [         4480477208          4484986024          4484988024
           4484986424          4484985464          4484986584
           4485083928          4485086568          4462815896
           4462817176          4461786920]
 [         4461788120          4461788680          4482633544
           4485086488          4482631944          4461787800
           4484998152          4484997592          4485046344
           4485075976          4484997512]
 [         4485076616          4485048824          4480529736
           4480464760          4462986008          4480475688
           4485077576          4461999912          4485075256
           4461998712          4461999272]
 [         4485046984          4462000632          4485141992
           44620005

In [410]:
nonpredictors = np.empty([a[0].size,1],int)
print(nonpredictors.shape)
for index, row in enumerate(a):
    order = get_order(row)
    print(index)
    nonpredictors[index] = np.count_nonzero(order-order_0)

(1, 1)


ValueError: XA must be a 2-dimensional array.

In [378]:
np.where(nonpredictors!=0)

(array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10]),
 array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]))

In [379]:
columns = columns [np.where(nonpredictors!=0)[0]]

In [380]:
columns

array([ 2,  4,  8, 11, 14, 35, 38, 42, 47, 48, 51])